In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `/scratch/users/merciyes18/Projects/latentplan.jl`


In [2]:
include("datasets/sequence.jl")
include("datasets/d4rl.jl")

No module named 'flow'
/kuacc/users/merciyes18/.julia/conda/3/lib/python3.9/site-packages/glfw/__init__.py:912: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: May 20 2022 19:45:31


Main.D4RL

In [3]:
using .Sequence: SequenceDataset
using .D4RL: load_environment, qlearning_dataset_with_timeouts

In [4]:
using Statistics: mean, std
using ProgressMeter: @showprogress

In [5]:
sequence_length::Int32=Int32(250)
step::Int32=Int32(10)
discount::Float32=0.99f0
max_path_length::Int32=Int32(1000)
device::String="cuda:0"
normalize_raw::Bool=true
normalize_reward::Bool=true
train_portion::Float32=1.0f0
disable_goal::Bool=false

false

In [6]:
env = load_environment("halfcheetah-medium-expert-v2")

/kuacc/users/merciyes18/.julia/conda/3/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


PyObject <d4rl.utils.wrappers.NormalizedBoxEnv object at 0x7f21e85acd60>

In [7]:
dataset = qlearning_dataset_with_timeouts(env.unwrapped, terminate_on_end=true, disable_goal=disable_goal, debug=true)

load datafile: 100%|█████████████████████████████| 9/9 [00:06<00:00,  1.32it/s]
Generating dataset 100%|█████████████████████████████████| Time: 0:00:01


Dict{String, Array} with 6 entries:
  "observations"      => Float32[0.0198319 -0.00384861 … -0.092561 -0.0935423; …
  "actions"           => Float32[-0.222937 -0.725157 … -0.627818 -0.994951; -0.…
  "terminals"         => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0…
  "realterminals"     => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0…
  "next_observations" => Float32[-0.00384861 -0.0552984 … -0.0935423 -0.0902453…
  "rewards"           => Float32[-0.20008 -1.19815 … 4.39841 4.58797]

In [8]:
observations = dataset["observations"]
actions = dataset["actions"]
rewards = dataset["rewards"]
terminals = dataset["terminals"]
realterminals = dataset["realterminals"]

49-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [9]:
obs_mean, obs_std = mean(observations, dims=2)  , std(observations, dims=2, corrected=false)
act_mean, act_std = mean(actions, dims=2), std(actions, dims=2, corrected=false)
reward_mean, reward_std = mean(rewards), std(rewards, corrected=false)

if normalize_raw
    observations = (observations .- obs_mean) ./ obs_std
    actions = (actions .- act_mean) ./ act_std
end

observations_raw = observations
actions_raw = actions
joined_raw = cat(observations, actions, dims=1) # join at the last dim
rewards_raw = rewards
terminals_raw = terminals

49-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [10]:
function segment(observations, terminals, max_path_length)
    @assert size(observations, 2) == size(terminals, 1)
    observation_dim = size(observations, 1)
    trajectories = [[]]

    for (obs, term) in zip(eachcol(observations), terminals)
        push!(trajectories[end], obs)
        if term
            push!(trajectories, [])
        end 
    end

    if length(trajectories[end]) == 0
        trajectories = trajectories[:end-1]
    end

    trajectories = [reduce(hcat, trajectory) for trajectory in trajectories]
    n_trajectories = length(trajectories)
    path_lengths = [size(traj, 2) for traj in trajectories]

    # pad trajectories to be equal length
    trajectories_pad = zeros(eltype(trajectories[1]), (observation_dim, max_path_length, n_trajectories))
    early_termination = zeros(Bool, (max_path_length, n_trajectories))
    for (i, traj) in enumerate(trajectories)
        path_length = path_lengths[i]
        trajectories_pad[:, 1:path_length, i] = traj
        early_termination[path_length+1:end, i] .= 1
    end
    return trajectories_pad, early_termination, path_lengths
end 

function compute_values(rewards_segmented::Array{Float32, 3}, discounts::Matrix{Float32}, max_path_length::Int32)
    values_segmented = zeros(Float32, size(rewards_segmented)...)
    @showprogress "Calculating values" for t in 1:max_path_length
        V = sum(rewards_segmented[:, t+1:end, :] .* discounts[:, 1:end-t], dims=2)
        values_segmented[:, t] = V
    end
    return values_segmented
end

compute_values (generic function with 1 method)

In [11]:
println("[ datasets/sequence ] Segmenting...")
joined_segmented, termination_flags, path_lengths = segment(joined_raw, terminals, max_path_length)
rewards_segmented, _, _ = segment(rewards_raw, terminals, max_path_length)

[ datasets/sequence ] Segmenting...


([-0.20008013 -1.1981525 … 0.0 0.0;;;], Bool[0; 0; … ; 1; 1;;], [49])

In [13]:
discounts = reshape(discount .^ collect(0:max_path_length-1), 1, :)

1×1000 Matrix{Float32}:
 1.0  0.99  0.9801  0.970299  0.960596  …  4.44932f-5  4.40482f-5  4.36077f-5

In [14]:
@time values_segmented = compute_values(rewards_segmented, discounts, max_path_length)

  0.008432 seconds (10.01 k allocations: 6.264 MiB)


1×1000×1 Array{Float32, 3}:
[:, :, 1] =
 90.3673  92.4903  94.1223  94.8021  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [17]:
dataset = SequenceDataset("halfcheetah-medium-expert-v2", penalty=Int32(-100))

[ datasets/sequence ] Sequence length: 250 | Step: 10 | Max path length: 1000
[ datasets/sequence ] Loading...
✓
[ datasets/sequence ] Segmenting...

load datafile: 100%|█████████████████████████████| 9/9 [00:06<00:00,  1.37it/s]



✓


SequenceDataset(PyObject <d4rl.utils.wrappers.NormalizedBoxEnv object at 0x7f2433d34220>, 250, 10, 1000, "cuda:0", false, true, true, Float32[-0.061123185; -0.033198655; … ; -0.02897517; -0.124439895;;], Float32[0.044988006; 0.14535953; … ; 4.9651423; 7.060827;;], Float32[-0.18401843; -0.5445832; … ; -0.29385233; -0.10082741;;], Float32[0.8482768; 0.55125207; … ; 0.67112625; 0.66300446;;], 2.4618914f0, 1.3244052f0, Float32[1.7994819 1.2731078 … -0.69880384 -0.72061616; -0.3873338 -0.13205646 … -0.025040638 0.9324437; … ; 0.019616777 -1.8649176 … -0.68722326 -0.7199059; -0.0019368734 0.6879222 … -1.4448111 0.25321737], Float32[-0.04588002 -0.63792706 … -0.5231779 -0.95597655; -0.34714535 -0.68787664 … -0.6692711 -0.79210204; … ; -0.8162434 -0.950165 … -0.27807504 1.0273684; 0.8071557 1.6280936 … -1.3067367 1.1480011], Float32[1.7994819 1.2731078 … -0.69880384 -0.72061616; -0.3873338 -0.13205646 … -0.025040638 0.9324437; … ; -2.0099375 -2.763538 … 1.4621775 1.6053095; 0.8907754 0.9625501